In [ ]:
import pandas as pd
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import shapely

In [ ]:
all_csvs = glob.glob('./out/sentinel_2021_v6_wgs84_combined_merged.csv')
all_csvs.sort()

In [ ]:
all_csvs

In [ ]:
def read_process_csv(csv):
    temp_df = pd.read_csv(csv)
    temp_df['satellite'] = os.path.basename(csv)[:8]
    temp_df['year'] = int(os.path.basename(csv)[9:13])
    return temp_df

In [ ]:

full_df = pd.concat([
    read_process_csv(csv) for csv in all_csvs
])

full_df = full_df.loc[full_df['hydropoly_max']<100]
full_df['area_ha'] = full_df['area']*100/10000 # HA
full_df['area_km'] = full_df['area']*100/(1000*1000) # km2
full_df = full_df.loc[full_df['area_ha']<100] # Remove less than 100 ha

In [ ]:

(full_df['area_ha'] <= 100).sum()

In [ ]:
full_df['area_km'].sum()

In [ ]:
lulc_df = pd.read_csv('../lulc/out/lulc_stats_res_sentinel_2021_summarized.csv')

In [ ]:
full_df[['center_lat','center_lon','latitude','longitude', 'area_ha']].rename(columns={'center_lat':'y_aea','center_lon':'x_aea'}).to_csv(
    './data/sentinel_2021_v6_cleaned.csv', index=False
)

In [ ]:

groupby_year_sat = full_df.groupby(['year', 'satellite']).agg(
    {'area_ha': ['mean','median','sum','count']}
)

In [ ]:
print(groupby_year_sat)

# Basic area distribution stats

In [ ]:
print(full_df.loc[(full_df.year==2021) & (full_df.area_ha > 10), 'area_km'].count())
print(full_df.loc[(full_df.year==2021) & (full_df.area_ha > 10), 'area_km'].count()/full_df.loc[full_df.year==2021].shape[0])
print(full_df.loc[(full_df.year==2021) & (full_df.area_ha < 1), 'area_km'].count()/full_df.loc[full_df.year==2021].shape[0])

In [ ]:
total_area = full_df.loc[full_df.year==2021, 'area_km'].sum()
print(full_df.loc[full_df.year==2021, 'area_km'].sum())
print(full_df.loc[full_df.year==2021, 'area_ha'].sum())
print(full_df.loc[(full_df.year==2021) & (full_df.area_ha > 10), 'area_km'].sum())
print(full_df.loc[(full_df.year==2021) & (full_df.area_ha > 10), 'area_km'].sum()/total_area)

print(full_df.loc[(full_df.year==2021) & (full_df.area_ha < 1), 'area_km'].sum())
print(full_df.loc[(full_df.year==2021) & (full_df.area_ha < 1), 'area_km'].sum()/total_area)

In [ ]:
plt.hist(full_df.loc[full_df['year'] == 2021, 'area_ha'], bins=np.arange(0, 5.01, 0.5))